In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from datetime import datetime

In [3]:
full_temperature = pd.read_csv("./raw/temperature.csv")

In [4]:
full_temperature

,time,G(i),H_sun,T2m,WS10m,Int
0,20060101:0010,0.0,0.0,3.39,4.28,0.0
1,20060101:0110,0.0,0.0,2.93,4.21,0.0
2,20060101:0210,0.0,0.0,2.65,4.28,0.0
3,20060101:0310,0.0,0.0,2.49,4.28,0.0
4,20060101:0410,0.0,0.0,2.12,4.21,0.0
...,...,...,...,...,...,...
43819,20101231:1910,0.0,0.0,-0.11,2.21,0.0
43820,20101231:2010,0.0,0.0,-0.27,2.21,0.0
43821,20101231:2110,0.0,0.0,-0.48,2.21,0.0
43822,20101231:2210,0.0,0.0,-0.68,2.00,0.0


In [5]:
temperature = full_temperature[['time', 'G(i)', 'H_sun', 'T2m', "WS10m"]].copy()
temperature.loc[:,'Datetime'] = pd.to_datetime(temperature.loc[:,'time'], format="%Y%m%d:%H%M")
temperature.set_index('Datetime', inplace=True)
temperature = temperature.loc[:, ['G(i)', 'H_sun', 'T2m', "WS10m"]]
temperature = temperature.resample('min').interpolate(method='linear')

In [10]:
temperature_episodes = temperature.groupby(pd.Grouper(freq='W'))

In [11]:
temperature_episodes.describe()

G(i)                                                     \
              count        mean         std  min  25%  50%         75%   
Datetime                                                                 
2006-01-01   1430.0   62.504895  112.963076  0.0  0.0  0.0   76.297250   
2006-01-08  10080.0   61.614643  132.603439  0.0  0.0  0.0   49.573958   
2006-01-15  10080.0   59.445119  141.744610  0.0  0.0  0.0   37.857958   
2006-01-22  10080.0   43.552143   94.139900  0.0  0.0  0.0   51.033000   
2006-01-29  10080.0  103.987738  192.188642  0.0  0.0  0.0  106.800750   
...             ...         ...         ...  ...  ...  ...         ...   
2010-12-05  10080.0   44.348988   89.535262  0.0  0.0  0.0   57.240250   
2010-12-12  10080.0   36.470298   80.628336  0.0  0.0  0.0   39.418417   
2010-12-19  10080.0   68.118155  149.557108  0.0  0.0  0.0   50.522083   
2010-12-26  10080.0   43.948333   92.086446  0.0  0.0  0.0   45.069292   
2011-01-02   7151.0   40.060887   82.772522  0.0  0.0  0.0   44.047250   

                      H_sun            ...       T2m           WS10m  \
               max    count      mean  ...       75%    max    count   
Datetime                               ...                             
2006-01-01  465.98   1430.0  4.048951  ...  5.857833   6.98   1430.0   
2006-01-08  635.37  10080.0  4.141726  ...  3.226667   6.00  10080.0   
2006-01-15  644.13  10080.0  4.029762  ...  4.838042   7.32  10080.0   
2006-01-22  584.41  10080.0  4.713571  ...  7.820208  10.21  10080.0   
2006-01-29  715.43  10080.0  5.435000  ...  1.104167   5.69  10080.0   
...            ...      ...       ...  ...       ...    ...      ...   
2010-12-05  475.57  10080.0  4.304107  ...  0.048000   3.05  10080.0   
2010-12-12  580.13  10080.0  4.104405  ...  2.834042   6.05  10080.0   
2010-12-19  660.02  10080.0  3.945655  ...  1.430000   5.27  10080.0   
2010-12-26  449.76  10080.0  3.902738  ...  1.658500   4.25  10080.0   
2011-01-02  461.75   7151.0  3.984114  ...  2.868250   5.20   7151.0   

                                                                           
                mean       std    min       25%       50%       75%   max  
Datetime                                                                   
2006-01-01  4.331871  0.349013  3.790  4.058917  4.280000  4.609208  5.24  
2006-01-08  2.352402  0.720666  0.690  1.736333  2.407167  2.900000  3.93  
2006-01-15  2.601118  0.717514  0.140  2.210000  2.714500  3.030000  4.76  
2006-01-22  3.125076  1.188388  0.620  2.386375  2.988000  3.607375  6.83  
2006-01-29  3.247420  1.062064  0.830  2.536000  3.268000  3.923000  6.48  
...              ...       ...    ...       ...       ...       ...   ...  
2010-12-05  3.068849  1.384091  0.140  2.063000  3.019167  4.088667  5.86  
2010-12-12  2.841426  0.734006  0.985  2.438000  2.827667  3.170000  5.72  
2010-12-19  3.666071  1.525715  0.690  2.641000  3.378167  4.256958  9.17  
2010-12-26  3.400781  1.705193  0.210  1.930000  3.056750  5.170000  6.97  
2011-01-02  2.530617  0.679324  1.030  2.083750  2.637500  3.046667  4.07  

[262 rows x 32 columns]

In [12]:
temperature_episodes = pd.concat([group.fillna(0) for name, group in temperature_episodes if len(group.dropna()) >= 10000 and group.index[0] >= datetime(2007, 1, 1) and group.index[-1] <= datetime(2008, 12, 31)]).groupby(pd.Grouper(freq='W'))

In [13]:
temperature_episodes.describe()

G(i)                                                           \
              count       mean         std  min  25%  50%        75%     max   
Datetime                                                                       
2007-01-07  10080.0  65.601488  126.029495  0.0  0.0  0.0  78.483208  618.24   
2007-01-14  10080.0  72.901964  139.213769  0.0  0.0  0.0  87.155417  642.31   
2007-01-21  10080.0  64.851488  139.854943  0.0  0.0  0.0  59.804167  684.38   
2007-01-28  10080.0  72.828095  138.521016  0.0  0.0  0.0  79.381750  655.98   
2007-02-04  10080.0  78.583869  164.590869  0.0  0.0  0.0  68.422167  728.06   
...             ...        ...         ...  ...  ...  ...        ...     ...   
2008-11-30  10080.0  59.858333  111.692302  0.0  0.0  0.0  72.911333  573.30   
2008-12-07  10080.0  56.248274  122.057863  0.0  0.0  0.0  57.231042  656.99   
2008-12-14  10080.0  41.824583   91.790991  0.0  0.0  0.0  49.379000  637.21   
2008-12-21  10080.0  41.731667   91.694894  0.0  0.0  0.0  52.564833  514.35   
2008-12-28  10080.0  97.954464  178.424766  0.0  0.0  0.0  95.260750  635.91   

              H_sun            ...        T2m            WS10m            \
              count      mean  ...        75%     max    count      mean   
Datetime                       ...                                         
2007-01-07  10080.0  4.109226  ...   9.970625  12.455  10080.0  4.923383   
2007-01-14  10080.0  4.269048  ...  11.346708  13.070  10080.0  4.812901   
2007-01-21  10080.0  4.766131  ...  12.646667  14.380  10080.0  5.432972   
2007-01-28  10080.0  5.215536  ...   3.180125   7.660  10080.0  3.401991   
2007-02-04  10080.0  6.003988  ...   8.403500  10.820  10080.0  2.373178   
...             ...       ...  ...        ...     ...      ...       ...   
2008-11-30  10080.0  4.633810  ...   4.051167   6.250  10080.0  2.871573   
2008-12-07  10080.0  4.286905  ...   5.767708   8.920  10080.0  3.678100   
2008-12-14  10080.0  4.047440  ...   2.596708   4.260  10080.0  2.578551   
2008-12-21  10080.0  3.921429  ...   7.846167  10.490  10080.0  2.870085   
2008-12-28  10080.0  3.911964  ...   6.322000   9.300  10080.0  3.421135   

                                                                     
                 std       min       25%       50%       75%    max  
Datetime                                                             
2007-01-07  1.193412  2.340000  4.163833  4.849333  5.652625   8.29  
2007-01-14  1.687651  0.620000  3.702000  4.885333  5.864042   8.97  
2007-01-21  2.450818  1.836667  3.217667  5.254000  7.576042  10.83  
2007-01-28  1.727101  0.550000  2.210000  2.779833  4.140000   7.79  
2007-02-04  1.117147  0.070000  1.660000  2.084000  2.970000   5.52  
...              ...       ...       ...       ...       ...    ...  
2008-11-30  0.778193  0.550000  2.340000  2.749500  3.311833   5.17  
2008-12-07  1.622709  1.030000  2.149250  3.454667  4.866708   7.86  
2008-12-14  1.173983  0.830000  1.706500  2.232917  2.971625   5.72  
2008-12-21  0.833122  0.550000  2.237833  2.799667  3.548000   4.62  
2008-12-28  1.342886  1.170000  2.210000  3.372500  4.480000   6.62  

[104 rows x 32 columns]

In [17]:
demand =  pd.HDFStore('./minutely/demand.h5')
with pd.HDFStore('./minutely/information.h5') as store:
    # Save each DataFrame with a key
    i = 0
    for name, group in temperature_episodes:
        if name.date() == demand[f"eps_{i}"].index[-1].date():
            store[f'eps_{i}'] = group
            i += 1

file = pd.HDFStore('./minutely/information.h5')
print(len(file) == len(demand))
file.close()
demand.close()